# 4 - Feras Formidáveis

## 4.13 A liga ternária mais cara do mundo

**Autores:** 

Enzo Januzzi Xavier

Rafael Anis Shaikhzadeh Santos

**Contribuição:** Ambos discutiram o problema e fizeram a maior parte do código juntos. Se dividiram apenas para fazer as funções objetivo e de mutação. Enzo fez a 'função objetivo' e Rafael a 'função de mutação'.

### Introdução:

O objetivo desta atividade é encontrar uma liga de três elementos que tenha o maior custo possível utilizando algoritmos genéticos. A liga ternária deve ser da forma **`x A . y B . z C`** sendo que 

$$
\begin{cases}
x + y + z = 100\,\mathrm{g} \\
x \geq 5\,\mathrm{g} \\
y \geq 5\,\mathrm{g} \\
z \geq 5\,\mathrm{g}
\end{cases}
$$

e *A*, *B* e *C* são elementos químicos diferentes. O preço de cada elemento disponível está definido abaixo. Considerou-se que qualquer composto com 3 elementos químicos é chamado de liga, desprezando equivalência química nas fórmulas.

### Desenvolvimento:

Importando as bibliotecas necessárias [1,2] e os scripts, baseados no material de aula do professor Daniel Cassar [3,4]:

In [1]:
import random
from functools import partial

from Scripts import funcao_objetivo_pop_liga
from Scripts import populacao_liga as cria_populacao
from Scripts import selecao_torneio_max as funcao_selecao
from Scripts import cruzamento_ordenado as funcao_cruzamento
from Scripts import mutacao_elemento as funcao_mutacao

Segue o dicionário de elementos e preços concedidos pela questão.

In [2]:
DIC_PRECO = {
    "H": 1.39, "He": 24, "Li": 85.6, "Be": 857, "B": 3.68, "C": 0.122, "N": 0.14, "O": 0.154, "F": 2.16, "Ne": 240,
    "Na": 3.43, "Mg": 2.32, "Al": 1.79, "Si": 1.7, "P": 2.69, "S": 0.0926, "Cl": 0.082, "Ar": 0.931, "K": 13.6,
    "Ca": 2.35, "Sc": 3460, "Ti": 11.7, "V": 385, "Cr": 9.4, "Mn": 1.82, "Fe": 0.424, "Co": 32.8, "Ni": 13.9,
    "Cu": 6, "Zn": 2.55, "Ga": 148, "Ge": 1010, "As": 1.31, "Se": 21.4, "Br": 4.39, "Kr": 290, "Rb": 15500,
    "Sr": 6.68, "Y": 31, "Nb": 85.6, "Mo": 40.1, "Tc": 100000, "Ru": 10600, "Rh": 147000, "Pd": 49500, "Ag": 521,
    "Cd": 2.73, "In": 167, "Sn": 18.7, "Sb": 5.79, "Te": 63.5, "I": 35, "Xe": 1800, "Cs": 61800, "Ba": 0.275,
    "La": 4.92, "Ce": 4.71, "Pr": 103, "Nd": 57.5, "Pm": 460000, "Sm": 13.9, "Eu": 31.4, "Gd": 28.6, "Tb": 658,
    "Dy": 307, "Ho": 57.1, "Er": 26.4, "Tm": 3000, "Yb": 17.1, "Lu": 643, "Hf": 900, "Ta": 312, "W": 35.3,
    "Re": 4150, "Os": 12000, "Ir": 56200, "Pt": 27800, "Hg": 30.2, "Tl": 4200, "Pb": 2, "Bi": 6.36, "Po": 49200000000000,
    "Ac": 29000000000000, "Th": 287, "Pa": 280000, "U": 101, "Np": 660000, "Pu": 6490000, "Am": 750000,
    "Cm": 160000000000, "Bk": 185000000000, "Cf": 185000000000,
}


Definimos algumas constantes do problema. Aqui é interessante observar que decidimos manter os valores de x, y e z constantes como 90, 5 e 5 gramas (somando 100 gramas). A ideia por trás é, que para achar a liga mais valiosa é lógico pegarmos a maior quantidade de elementos mais valiosos. Nesse caso, o peso máximo possível de um mesmo elemento é 90 gramas - definido pelo enunciado (100-5-5 = 90). Os outros dois elementos terão 5 gramas, visto que é o mínimo necessário.
Importante destacar que essa lógica funciona neste problema, pois é um problema monoobjetivo - queremos apenas otimizar o valor da liga.


Nota: já que os valores dos elementos são dado em dolár/kg vamos por x,y,z em kg também.

In [3]:
ELEMENTOS = list(DIC_PRECO.keys())
TAMANHO_LIGA = 3
XYZ = [0.09, 0.005, 0.005]

NUM_GERACOES = 1000
TAMANHO_POPULACAO = 100
CHANCE_DE_CRUZAMENTO = 0.5
CHANCE_DE_MUTACAO = 0.025
CHANCE_DE_MUTACAO_TAMANHO = 0.05
TAMANHO_TORNEIO = 3

Alteramos nossa função objetivo usando partial.

In [4]:
funcao_objetivo = partial(funcao_objetivo_pop_liga, 
                          elementos_possiveis = ELEMENTOS, 
                          dic_precos = DIC_PRECO, 
                          xyz = XYZ)

Criando a população inicial do problema, em que cada indivíduo contém 3 elementos diferentes (A, B, C). Vale lembrar, de novo, que os valores de x,y e z são fixos.

In [5]:
populacao = cria_populacao(TAMANHO_POPULACAO, TAMANHO_LIGA, ELEMENTOS)
populacao

[['Ni', 'Ac', 'Gd'],
 ['Kr', 'Pr', 'Mg'],
 ['Ag', 'Pd', 'N'],
 ['Nb', 'Gd', 'Br'],
 ['Ir', 'Sm', 'Er'],
 ['Ce', 'K', 'Bi'],
 ['Ar', 'C', 'Ru'],
 ['Sc', 'Ge', 'Gd'],
 ['Mo', 'Ga', 'Hf'],
 ['K', 'Er', 'Cu'],
 ['Co', 'Rh', 'Po'],
 ['Lu', 'Ag', 'C'],
 ['O', 'Ti', 'Sr'],
 ['Bi', 'Ar', 'Pt'],
 ['Mo', 'Al', 'Cu'],
 ['Cr', 'Be', 'Pb'],
 ['Cf', 'Ba', 'Se'],
 ['Nb', 'Ga', 'Ba'],
 ['Sc', 'Be', 'Tm'],
 ['Re', 'Tl', 'Cs'],
 ['In', 'Cm', 'Se'],
 ['Zn', 'Ir', 'Pt'],
 ['Rh', 'Tc', 'Nd'],
 ['Fe', 'Tm', 'Ca'],
 ['Be', 'N', 'Sb'],
 ['Ne', 'Te', 'Nb'],
 ['Cr', 'Yb', 'Np'],
 ['Xe', 'Co', 'Ti'],
 ['Bk', 'Ho', 'Tb'],
 ['Ti', 'S', 'Os'],
 ['Tb', 'Zn', 'Bk'],
 ['Hg', 'He', 'Sr'],
 ['Cu', 'Mo', 'U'],
 ['Fe', 'Pm', 'Mn'],
 ['K', 'Ag', 'Mg'],
 ['Xe', 'H', 'Ru'],
 ['Pt', 'Ac', 'Hf'],
 ['Ce', 'Sb', 'Gd'],
 ['Yb', 'Ar', 'Rb'],
 ['Ho', 'Tl', 'I'],
 ['Ac', 'P', 'Gd'],
 ['Cf', 'Xe', 'Mn'],
 ['Bi', 'H', 'Co'],
 ['Hg', 'Pu', 'Mg'],
 ['Ag', 'Ni', 'Eu'],
 ['He', 'B', 'Al'],
 ['Cd', 'Tb', 'P'],
 ['Pb', 'Mo', 'P'],
 ['Cl', '

O código principal, contendo um laço while para fazer todas as etapas do algoritmo genético (seleção, cruzamento, mutação e atualização da geração) até encontrar a liga mais cara. A função objetivo calcula o peso total dos elementos da população, fazendo uma seleção por torneio buscando maximizar o fitness. Usou-se o cruzamento ordenado e a mutação de salto para alterar os elementos de cada liga, buscando encontrar o melhor candidato.

In [6]:
hall_da_fama = []

for n in range(NUM_GERACOES):
    
    # Seleção
    fitness = funcao_objetivo(populacao)        
    selecionados = funcao_selecao(populacao, fitness, TAMANHO_TORNEIO)
    
    # Cruzamento
    proxima_geracao = []
    for pai, mae in zip(selecionados[::2], selecionados[1::2]):
        individuo1, individuo2 = funcao_cruzamento(pai, mae, CHANCE_DE_CRUZAMENTO)
        proxima_geracao.append(individuo1)
        proxima_geracao.append(individuo2)
    
    # Mutação
    funcao_mutacao(proxima_geracao, CHANCE_DE_MUTACAO, ELEMENTOS)
    
    # Atualização do hall da fama
    fitness = funcao_objetivo(proxima_geracao)
        
    maior_fitness = max(fitness)
    indice = fitness.index(maior_fitness)
    hall_da_fama.append(proxima_geracao[indice])    
    
    # Encerramento
    populacao = proxima_geracao

Por fim, só nos resta identificar o melhor individuo observado, basta analisar o individuo com maior fitness no hall da fama.

In [7]:
fitness = funcao_objetivo(hall_da_fama)
maior_fitness = max(fitness)
indice = fitness.index(maior_fitness)
melhor_individuo_observado = hall_da_fama[indice]


print("O melhor indivíduo observado foi:")
print("Elementos:", melhor_individuo_observado)
print ('Pesos respectivo (kg):', XYZ)

print("Valor da liga (Doláres):", maior_fitness)

O melhor indivíduo observado foi:
Elementos: ['Po', 'Ac', 'Bk']
Pesos respectivo (kg): [0.09, 0.005, 0.005]
Valor da liga (Doláres): 4573925000000.0


### Conclusão:

Ao fim, conseguimos uma liga terciária que acreditamos ser a mais valiosa. Apesar de não fazermos uma busca exaustiva para checar essa resposta, logicamente parece ser um bom palpite também. Afinal, pegamos 90 gramas do elemento mais caro e 5 gramas dos segundo e terceiro elemento mais caro.

### Referências:

[1] Biblioteca Random. https://docs.python.org/3/library/random.html

[2] Biblioteca Functools. **Método Partial**. https://docs.python.org/3/library/functools.html#partial-objects

[3] CASSAR, Daniel. "ATP-303 GA 6.2 - Notebook do problema da mochila.ipynb". Material de Aula, 2025.

[4] CASSAR, Daniel. "funcoes.py". Scripts baseados, 2025

